In [1]:
#Environment set up
import os
os.environ["OPENAI_API_KEY"] = ...

In [ ]:
# pip install numexpr google-search-results

In [2]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, ConversationChain
from langchain.agents import load_tools, initialize_agent,AgentType

In [3]:
#load the language model we're going to use to control the agent.
llm_OpenAI = OpenAI(temperature=0.8)

In [4]:
# load some tools e.g. Database lookup, Python REPL, other chains. 
#The interface for a tool is currently a function that is expected to have a string as an input..
#..with a string as an output. that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm_OpenAI)

In [5]:
# initialize an agent with the tools, the language model, and the type of agent we want to use.
# agents dynamically Call Chains Based on User Input
# to specify the agent required, we give a string that references a support agent class.
Project_agent = initialize_agent(tools, llm_OpenAI, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [6]:
# run
output0=Project_agent.run("what might be costs for building a new school in Salisbury, England? thanks")



> Entering new AgentExecutor chain...
 I should do some research about the cost of building in Salisbury
Action: Search
Action Input: building cost Salisbury England
Observation: Search our selection of properties for sale in Salisbury. ... Guide price £122,500 -​ £152,500 ... Alderbury, Salisbury, Wiltshire, SP5 3AS.
Thought: That's useful but I need more information about the cost of a new school building
Action: Search
Action Input: cost of building new school Salisbury England
Observation: The total cost of new block and demolition of old one is £4.545m, of which £1.545m comes from a Department for Education (DfE) grant and £3m is ...
Thought: I now know the estimated cost of building a new school in Salisbury, England
Final Answer: The estimated cost of building a new school in Salisbury, England is approximately £4.545 million.

> Finished chain.


Memory: Add State to Chains and Agents
So far, all the chains and agents we’ve gone through have been stateless. 
the ConversationChain is one of several options with two different types of memory.
By default, the ConversationChain all previous inputs/outputs and adds them to the context that is passed.

In [8]:
Project_reviewer = ConversationChain(llm=llm_OpenAI, verbose=True)
output1 = Project_reviewer.predict(input="Hi. You are reviewing the proposed costs for a school in Salisbury. Please state a plan for improving the accuracy of these costs. Here they are:"+output0)
print(output1)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi. You are reviewing the proposed costs for a school in Salisbury. Please state a plan for improving the accuracy of these costs. Here they are:The estimated cost of building a new school in Salisbury, England is approximately £4.545 million.
AI:

> Finished chain.
 Hi! My plan for improving the accuracy of these costs involves gathering more detailed information. Specifically, I would need to know the total square feet of the proposed school, the number of classrooms, the number of other amenities to be included, such as sports fields or a cafeteria, and an estimate of the cost of labor and materials in the area. With this information, I can provide a much mo

In [9]:
output2 = Project_reviewer.predict(input="Thankyou. Please could you break that down the first task")
print(output2)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi. You are reviewing the proposed costs for a school in Salisbury. Please state a plan for improving the accuracy of these costs. Here they are:The estimated cost of building a new school in Salisbury, England is approximately £4.545 million.
AI:  Hi! My plan for improving the accuracy of these costs involves gathering more detailed information. Specifically, I would need to know the total square feet of the proposed school, the number of classrooms, the number of other amenities to be included, such as sports fields or a cafeteria, and an estimate of the cost of labor and materials in the area. With this information, I can provide a much more accurate estimate

In [4]:
##pass few shot examples to a prompt template
from langchain import PromptTemplate, FewShotPromptTemplate

# First, create the list of few shot examples.
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

In [2]:
# Next, we specify the template to format the examples we have provided.
# We use the `PromptTemplate` class for this.
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""

In [6]:
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

In [7]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the antonym of every input",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Word: {input}\nAntonym:",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n\n",
)

In [8]:
# We can now generate a prompt using the `format` method.
print(few_shot_prompt.format(input="big"))


Give the antonym of every input


Word: happy
Antonym: sad




Word: tall
Antonym: short



Word: big
Antonym:
